# Building Graphs in wc_rules

In `wc_rules`, we are essentially constructing graph representations of chemical entities. The basic building blocks of the graph are 
* nodes, which include molecules, sites and site-relations, and,
* edges, which are bidirectional relations between molecules, sites and site-relations. 

To build this graph, one needs to know about :
* subclasses
* instances
* relatedness management
* attribute querying

`wc_rules` uses Python class inheritance to dictate types of molecules, sites and site-relations, and allows the modeler to freely subclass any of the provided types. For example, let us create the type hierarchy `Molecule -> Receptor -> EGFR`

In [1]:
from wc_rules.chem2 import Molecule,Site

class Receptor(Molecule):
    pass

class EGFR(Receptor):
    pass

To create instances of classes (nodes on the graph), simply call an empty constructor.

In [2]:
x = EGFR()
x

Class inheritance allows instances to be type-checked against any parent class.

In [3]:
isinstance(x,EGFR) and isinstance(x,Receptor) and isinstance(x,Molecule)

True

Instance attributes often have `get_<>()` and `set_<>()` methods to allow access and instantiation. For example, to set and get the `id` attribute on instance `x`, we have `set_id(id)` and `get_id()`. The `set_<>()` methods always return the instance from which they are called.

In [4]:
x.set_id('egfr_instance_001')

In [5]:
x.get_id()

'egfr_instance_001'

Instance attributes also maintain the edges of the graph, a.k.a., _relations_. Typically a pair of attributes are involved in maintaining a particular type of relation between two classes. The convention we use is as follows:

* If an instance of a class has a particular relation to many instances of another class, 
    * we use an attribute name that is plural,
    * we use `add_<>s()` and `remove_<>s()` to manage these relations,
    * we use `get_<>s()` to access the related instances.

For example, a `Molecule` instance may be related to many `Site` instances, so it has an attribute `sites`, which holds a list of references to `Site` instances, and is managed using `add_sites()`, `remove_sites()` and `get_sites()`.

* If an instance of a class has a particular relation to only a single instance of another class,
    * we use an attribute name that is singular,
    * we use `set_<>()` and `unset_<>()` to manage the relation,
    * we use `get_<>()` to access the related instance.

For example, a `Site` instance may be related to only one `Molecule` instance, so it has an attribute `molecule`, which holds a reference to a `Molecule` instance, and is managed by `set_molecule()`, `unset_molecule()` and `get_molecule()`.

A relation can be set on either end, e.g., `Molecule.add_sites()` and `Site.set_molecule()` have the same outcome in creating a relation between a molecule and a site.

In [6]:
# Using add_sites() to add s1 to m
m = Molecule().set_id('m')
s1 = Site().set_id('s1')

m.add_sites(s1)
m.get_sites()

In [7]:
# Using set_molecule() to add s2 to m
s2 = Site().set_id('s2')
s2.set_molecule(m)
s2.get_molecule()

In [8]:
# m should have two sites now
m.get_sites()

The `add/remove/set/unset` methods all return the original object from which they are called and so can be chained.

In [9]:
# Adding 3 more sites, then chaining and adding 1 more site
m.add_sites( Site(), Site(), Site() ).add_sites( Site() )

# m should have 6 sites now
m.get_sites()

If a `get_` method returns a list of instances, it can typically be used to also filter over that list, e.g.,

In [10]:
# filtering using the id attribute
m.get_sites(id='s1')